In [1]:
!pip install efficientnet -q

In [2]:
import os, math
import psutil, random

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import albumentations as A
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import cv2; print(cv2.__version__)
import tensorflow as tf; print(tf.__version__)

4.5.2
2.4.1


In [3]:
import itertools
import os
import random
import gc
import numpy as np
import pandas as pd
import math
import pprint
import matplotlib.pylab as plt
import seaborn as sns
sns.set(rc={"axes.titlesize":15, "axes.labelsize":9,"axes.titlepad":15,
            "axes.labelpad":12, "legend.fontsize":9,
            "legend.title_fontsize":9, "figure.titlesize":15,
            "axes.grid":False})

from sklearn.model_selection import train_test_split, GroupKFold
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets

print('TF version:', tf.__version__)
print('Hub version:', tfhub.__version__)
print('Physical devices:', tf.config.list_physical_devices())

TF version: 2.4.1
Hub version: 0.12.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Data Preprocess

In [4]:
study_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv'); print(study_df.shape)
study_df['StudyInstanceUID'] = study_df['id'].apply(lambda x: x.replace('_study', ''))
del study_df['id']

def hot_to_sparse(row):
    return(row.index[row.apply(lambda x: x==1)][0])
study_df['diagnosis'] = study_df.apply(lambda row:hot_to_sparse(row), axis=1)
cls = {
    'Typical Appearance':1,                    
    'Negative for Pneumonia':0,                
    'Indeterminate Appearance':2,                     
    'Atypical Appearance':3,    
}
study_df['sparse_gt'] = study_df.diagnosis.map(cls) 

image_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv'); print(image_df.shape)
train = image_df.merge(study_df, on='StudyInstanceUID')
train['id'] = train['id'].apply(lambda x: x.replace('_image', ''))
display(train.head()); print(train.shape)

(6054, 5)
(6334, 4)


,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,diagnosis,sparse_gt
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,Typical Appearance,1
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0,Negative for Pneumonia,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0,Typical Appearance,1
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1,Atypical Appearance,3
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0,Typical Appearance,1


(6334, 10)


In [5]:
train = train[['id', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance', 'sparse_gt']]
train.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt
0,000a312787f2,0,1,0,0,1
1,000c3a3f293f,1,0,0,0,0
2,0012ff7358bc,0,1,0,0,1
3,001398f4ff4f,0,0,0,1,3
4,001bd15d1891,0,1,0,0,1


In [6]:
duplicates = ['11f46d9df786',
 'ba57c39bc15d',
 'bd4b6cfeddee',
 '82c84d5079b7',
 'a27a126be5c1',
 '3d12cb6aad8b',
 'b814a1e29517',
 '09236f094647',
 '36b995a9575c',
 '9cdc2af4fbb0',
 '779f0040d1b2',
 'ef6e312ca719',
 'b97c6b32105e',
 '93301812b0e7',
 '4c414b793562',
 'e6cc65d9de1d',
 '9fcbe25a88e0',
 'f228350bd770',
 '59bd5a00b33e',
 'fa5e3f933004',
 'a8e019dcf085',
 'd205dc170ccc',
 'a5a364383f34',
 '9939f63af4ff',
 '646a6a0a9bf2',
 '581fd56bc4bf',
 '629c9a257b43',
 '2ebe16a053fe',
 '173c23887f9b',
 'ea2117b53323',
 '57bf106a9e2e',
 'b989acc3c046',
 '12fe8eadded1',
 '7f9c98b88aec',
 'c00c7e8b673c',
 'a5bbd30ed109',
 'b622f84403da',
 'ba375ad92c5f',
 'b0fd33e251f5',
 'c6e92e59a0ae',
 'ef690fe6f288',
 '6970c59656c9',
 'ab55abb953d1',
 'd540d6c2a425',
 '262dc03560d6',
 'fc44ab5de22c',
 'ce51b397b1a6',
 '6e9fad584bff',
 'a2429a60f3ea',
 'b0866caa201a',
 '7e4a4ef3ff9f',
 '3577ee4f26c4',
 '6f54e9cbd180',
 '57faddfc733e',
 '6a93346150a4',
 '611348a721f7',
 'c4b2940d2334',
 'e007bc90c441',
 '3cde0ddf01c7',
 '296aaf716ae0',
 'b7eb26ddf5a3',
 'e738c549fe8e',
 '39d52f244db3',
 'a9704ce54780',
 '1701be774ecb',
 '221ff464c9fc',
 'e3f8e3a3baa3',
 'be65d1a22de5',
 '103bd9ed3929',
 '9108cdfd43dc',
 '3d2224cb6bb7',
 'c4b68b29a072',
 '6d36ffbc7864',
 '35da4a5a0201',
 '341787fb81c7',
 '6ff218d10741',
 '68fcf433c6fe',
 '25400558ba1e',
 '149356f04849',
 '39cd89772e5e',
 '99f3642f50f5',
 'c72e26e9ecae',
 '4b310bbd3cb1',
 'f208dc529d16',
 '4024e564722b',
 'cc708f2aaffe',
 'dcb374d8387e',
 '7f1924880cf8',
 '961f9c368664',
 '1ea01196514a',
 '725328925e0f',
 '35ea2c9841c0',
 '72e5fbd61953',
 '75b52bec817f',
 'd27bca58e388',
 '44d7b2e7b0f9',
 '830063223a31',
 '0b020a7aff0a',
 '9d8ccbfb99b7',
 '082d3af6ecc0',
 'eef757610245',
 'b77430575510',
 '321c93d6503f',
 '0a990c89256a',
 'c9a028b48504',
 'c636ac67c19a',
 '76c66ee8e58d',
 'cdd9e3aaf45a',
 '9a9d89ca4aa0',
 '8b55ea197c78',
 '30b18db28900',
 'a57bf6dd6119',
 'e4a78ff8fa73',
 'e01d23a04187',
 '66dabc6f972d',
 '4cebe94f1fa3',
 '6792b9b0329d',
 'ddff141123c1',
 '550f057ee0b0',
 '0102b5cac730',
 'e7e072c73e1d',
 '00e3a7e91a34',
 '8cbf85033da8',
 '6f749e2783e1',
 '4b2f144a1a8b',
 '783b1efb5675',
 'e96133d06736',
 '869476b0763a',
 'e496735126c2',
 '00c1515729a8',
 '9a335ce0bca4',
 '9b1de1c45491',
 'd9456aadecbe',
 '51d04316f2c9',
 'fb1e24bf6698',
 'b3ffe59e37c7',
 '16bd0ff2f0cd',
 '8d4b3609ed92',
 'a174a4f6c807',
 '2a7a456d367e',
 '1b92142f4362',
 'bfefe70b56ca',
 'e5fc2fa92e65',
 '8a38c00672d5',
 'a94171e98807',
 '61d0f2ca0ddd',
 '0bd6cd815ba9',
 '1de0ad5314cc',
 'e897ef5c203c',
 '0acec7010081',
 '3b6ad60071d4',
 'b61f3493c551',
 '267a250932bc',
 'a8013eaf16c1',
 'e704aebd4ca5',
 '04cc2f7f4c4b',
 '3e7b2ffc97db',
 'e0e1e1462e56',
 'b5aa6189d142',
 'cbf0a27f993e',
 'ea2688741043',
 '2204baf77803',
 '56ff7c4e1f41',
 '156cb1f5c689',
 '05c063f5cef5',
 'c3a09e8a600d',
 'f577c6fb0f49',
 '8093df07a5d0',
 '6728e11290af',
 'efc93a3917b6',
 'c843d08b49b8',
 '717ea5155b46',
 '343d445230a1',
 '50f29acd37a6',
 'ea516e218fe6',
 '582c442e440b',
 'f9a92f1ba934',
 'bdd3115879aa',
 '68ad4b624a6d',
 'a323f1604ae2',
 'bb52c80c7f45',
 '0b858129adb4',
 '26c6c25e9467',
 '91f72df96e13',
 '19c6397cdec9',
 '6861c5992717',
 'deea6d6f81a5',
 '106632672b95',
 '6534a837497d',
 '684230477525',
 'df3d32c8df06',
 '3f0879b045e7',
 '505693d77a18',
 'a39667fe9a81',
 '2c130ee08736',
 'c9a9e5e120bd',
 'bb36f3d3ac71',
 '1c96d9b08487',
 'daa9bddc5fe6',
 'e73f2cc047ad',
 'e60a47d36e62',
 '6366191bfe49',
 'd180fed57716',
 '866e3622cb24',
 '55839276a9c9',
 'fa3a7fe547ca',
 '59bc532be971',
 '424118f1afc8',
 '6ddf030ec946',
 '3d0bac303fc8',
 'f689d30c2f86',
 '8c7bdccb65ed',
 'df4f1240317e',
 '25ff5d625b35',
 '75f207c4945d',
 '35e398a5a431',
 '5d395fff6278',
 'e3e2f20e0264',
 '99e8636292c9',
 'de94a43e947f',
 'a5beac3d9fb4',
 'd19ed8d811ad',
 'bf2417586cf2',
 '72cf260ddf4c',
 'f5451a98d684',
 'aad814e52627',
 '32222cc776a2',
 '608d574388ba',
 '9872a8a48f23',
 'f7f759f7a4f1',
 '88e7702b49c4',
 'dfd9078a4c6d',
 'd4131bd4e64a',
 '1a0e966f2d57',
 '9f50edab452b',
 '4cf41c062d0c',
 '13bf9d719ad1',
 '0eb641cb0dcd',
 'a2ee4b862182',
 '61e618dc7f9a',
 '467f412db5c4',
 '990216aa9f08',
 '3d047361508c',
 '08acae0bf785',
 '0dd32589bdc3',
 '9f180629878a',
 '6e5946091b8a',
 'f3ce7a4cfd20',
 'f85fe770048a',
 'cf0cdff4ef2e',
 'df2bb22fa871',
 '0d4d6acc9ed3',
 '45ff2dfff073',
 'd54f6204b044',
 'eea3a910fa9e',
 '3d19da0d4ee6',
 'c3510a436bff',
 '4cbc17936e7d',
 'b121806162c3',
 'e5380eb2d925',
 '2f973757eaa4',
 'b5a415f70aa9',
 'cc69791740bb',
 'a1fa5f79671d',
 'dbc311ed04b5',
 '744427c7dff8',
 '49664f078f0e',
 'bee62c601ae9',
 'e441b1fb74ee',
 '8df453b7557c',
 '7626c521cad7',
 '33a72bfb5ddb',
 '56ed68c28819',
 '04a0b90d7875',
 '08932dca1447',
 'daae293a8b40',
 'c0400d0e413d',
 'ddb051c1233b',
 'f13f9ace0a9a',
 '9c0633ef606b']

In [7]:
len(duplicates)

280

In [8]:
all_files = list(train.id.values)

In [9]:
for i in duplicates:
    all_files.remove(i)

In [10]:
len(all_files)

6054

In [11]:
final_df = pd.DataFrame({'id':all_files})

In [12]:
train_df = pd.merge(final_df, train, how='left', on='id')

In [13]:
len(train_df)

6054

In [14]:
train_df.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt
0,000a312787f2,0,1,0,0,1
1,000c3a3f293f,1,0,0,0,0
2,0012ff7358bc,0,1,0,0,1
3,001398f4ff4f,0,0,0,1,3
4,001bd15d1891,0,1,0,0,1


In [15]:
train_df.sum()

id                          000a312787f2000c3a3f293f0012ff7358bc001398f4ff...
Negative for Pneumonia                                                   1676
Typical Appearance                                                       2855
Indeterminate Appearance                                                 1049
Atypical Appearance                                                       474
sparse_gt                                                                6375
dtype: object

## VinBig

In [16]:
vin = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
vin.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [17]:
vin.shape

(67914, 8)

In [18]:
vin = vin.iloc[:,:4]

In [19]:
vin[vin.duplicated()]

,image_id,class_name,class_id,rad_id
575,d7aea6f37597df566725da73655edcc2,Pleural thickening,11,R10
1303,50708867ca7d310f7a521c99e8404366,Pleural effusion,10,R10
1344,ce5e76544b8c1d0ec34783b0f1bc471d,Nodule/Mass,8,R9
1399,ab11a974837f5313912804939bfae79e,Nodule/Mass,8,R8
1405,bc2be005526db7ab9d5ec6741ddee945,Pleural thickening,11,R10
...,...,...,...,...
67897,857b9d89572e03adb17d0630b33709ea,Pleural thickening,11,R9
67899,3b887c0550e9722c9b86b7c22ad09e11,Other lesion,9,R10
67906,26d1d5a0ef2e692c6340e74859ffdc53,Pulmonary fibrosis,13,R10
67907,22672ab82c290c20b86863291e25ef6c,ILD,5,R9


In [20]:
vin[vin['image_id'] == 'd7aea6f37597df566725da73655edcc2']

,image_id,class_name,class_id,rad_id
37,d7aea6f37597df566725da73655edcc2,Infiltration,6,R8
569,d7aea6f37597df566725da73655edcc2,Pleural thickening,11,R10
575,d7aea6f37597df566725da73655edcc2,Pleural thickening,11,R10
5560,d7aea6f37597df566725da73655edcc2,Infiltration,6,R10
14003,d7aea6f37597df566725da73655edcc2,Cardiomegaly,3,R10
17940,d7aea6f37597df566725da73655edcc2,Pleural effusion,10,R8
23221,d7aea6f37597df566725da73655edcc2,Aortic enlargement,0,R10
31900,d7aea6f37597df566725da73655edcc2,Infiltration,6,R9
36681,d7aea6f37597df566725da73655edcc2,Aortic enlargement,0,R9
39794,d7aea6f37597df566725da73655edcc2,Lung Opacity,7,R10


In [21]:
vin.drop_duplicates(inplace=True)

In [22]:
vin.shape

(61274, 4)

## Getting Atypical Data (External Sources (NIH and VinBig))

In [23]:
needed_samples = 2855
we_have = 474
extra_needed = needed_samples - we_have
extra_needed

2381

In [24]:
categories = 4
per_cat = extra_needed / categories
per_cat

595.25

## Vin_Big

In [25]:
vin_labels = vin

In [26]:
vin_labels.class_name.value_counts()

No finding            31818
Aortic enlargement     7157
Cardiomegaly           5414
Pulmonary fibrosis     3267
Pleural thickening     3228
Pleural effusion       2105
Lung Opacity           2036
Other lesion           1623
Nodule/Mass            1412
Infiltration            949
Calcification           686
ILD                     609
Consolidation           505
Atelectasis             266
Pneumothorax            199
Name: class_name, dtype: int64

In [27]:
vin_labels = vin_labels[['image_id', 'class_name']]
vin_labels['dataset'] = 'vinbig'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
list_cat = ['Pneumothorax', 'Pleural effusion', 'Consolidation', 'Nodule/Mass']
Pneumothorax_vin = vin_labels[vin_labels.class_name == 'Pneumothorax']
Pleural_effusion_vin = vin_labels[vin_labels.class_name == 'Pleural effusion']
Consolidation_vin = vin_labels[vin_labels.class_name == 'Consolidation']
Nodule_Mass_vin = vin_labels[vin_labels.class_name == 'Nodule/Mass']
Pneumothorax_vin.shape, Pleural_effusion_vin.shape, Consolidation_vin.shape, Nodule_Mass_vin.shape

((199, 3), (2105, 3), (505, 3), (1412, 3))

## NIH

In [29]:
# nih_labels = pd.read_csv('../input/data/Data_Entry_2017.csv')
# nih_labels.head()

In [30]:
# nih_labels = nih_labels[['Image Index', 'Finding Labels']]
# nih_labels.columns = ['image_id', 'class_name']

In [31]:
# Pneumothorax_nih = nih_labels[nih_labels['class_name'] == 'Pneumothorax']
# Consolidation_nih = nih_labels[nih_labels['class_name'] == 'Consolidation']
# Edema_nih = nih_labels[nih_labels['class_name'] == 'Edema']
# Nodule_nih = nih_labels[nih_labels['class_name'] == 'Nodule']
# Mass_nih = nih_labels[nih_labels['class_name'] == 'Mass']

In [32]:
# Pneumothorax_nih.shape, Consolidation_nih.shape, Edema_nih.shape, Nodule_nih.shape, Mass_nih.shape

## Adding Data

In [33]:
Pneumothorax_vin.head()

,image_id,class_name,dataset
212,6e4391555899c8474c4d32f42b2ba21b,Pneumothorax,vinbig
308,e3c5ff1eb0d55039f3bbc3c4875543c3,Pneumothorax,vinbig
1090,54e6184c63c75a9695d7effc17969ad0,Pneumothorax,vinbig
1140,71511f823c4664475131c1519d9bd0a8,Pneumothorax,vinbig
1440,f9a0f4bd4bb3553af6e939362bbba5d6,Pneumothorax,vinbig


In [34]:
Pneumothorax_vin.shape, Pleural_effusion_vin.shape, Consolidation_vin.shape, Nodule_Mass_vin.shape

((199, 3), (2105, 3), (505, 3), (1412, 3))

In [35]:
extra_needed

2381

In [36]:
per_cat

595.25

In [37]:
199+505+838+839

2381

In [38]:
Pleural_effusion_vin = Pleural_effusion_vin.sample(n=839, random_state=9)
Nodule_Mass_vin = Nodule_Mass_vin.sample(n=838, random_state=9)

In [39]:
Atypical_extra = pd.concat([Pneumothorax_vin, Pleural_effusion_vin, Consolidation_vin, Nodule_Mass_vin])
Atypical_extra.shape

(2381, 3)

In [40]:
Atypical_extra['Atypical Appearance'] = 1
Atypical_extra['sparse_gt'] = 3
Atypical_extra.head()

,image_id,class_name,dataset,Atypical Appearance,sparse_gt
212,6e4391555899c8474c4d32f42b2ba21b,Pneumothorax,vinbig,1,3
308,e3c5ff1eb0d55039f3bbc3c4875543c3,Pneumothorax,vinbig,1,3
1090,54e6184c63c75a9695d7effc17969ad0,Pneumothorax,vinbig,1,3
1140,71511f823c4664475131c1519d9bd0a8,Pneumothorax,vinbig,1,3
1440,f9a0f4bd4bb3553af6e939362bbba5d6,Pneumothorax,vinbig,1,3


# Getting Indeterminate Data (External Sources (NIH and VinBig))

In [41]:
needed_samples = 2855
we_have = 1049
extra_needed = needed_samples - we_have
extra_needed

1806

In [42]:
categories = 4
per_cat = extra_needed / categories
per_cat

451.5

## VinBig

In [43]:
vin_labels.class_name.value_counts()

No finding            31818
Aortic enlargement     7157
Cardiomegaly           5414
Pulmonary fibrosis     3267
Pleural thickening     3228
Pleural effusion       2105
Lung Opacity           2036
Other lesion           1623
Nodule/Mass            1412
Infiltration            949
Calcification           686
ILD                     609
Consolidation           505
Atelectasis             266
Pneumothorax            199
Name: class_name, dtype: int64

In [44]:
ILD_vin = vin_labels[vin_labels.class_name == 'ILD']
Atelectasis_vin = vin_labels[vin_labels.class_name == 'Atelectasis']
Infiltration_vin = vin_labels[vin_labels.class_name == 'Infiltration']
Pulmonary_fibrosis_vin = vin_labels[vin_labels.class_name == 'Pulmonary fibrosis']
ILD_vin.shape, Atelectasis_vin.shape, Infiltration_vin.shape, Pulmonary_fibrosis_vin.shape

((609, 3), (266, 3), (949, 3), (3267, 3))

## Adding Data

In [45]:
ILD_vin.shape, Atelectasis_vin.shape, Infiltration_vin.shape, Pulmonary_fibrosis_vin.shape

((609, 3), (266, 3), (949, 3), (3267, 3))

In [46]:
extra_needed

1806

In [47]:
per_cat

451.5

In [48]:
513+513+514+266

1806

In [49]:
ILD_vin = ILD_vin.sample(n=513, random_state=9)
Infiltration_vin = Infiltration_vin.sample(n=513, random_state=9)
Pulmonary_fibrosis_vin = Pulmonary_fibrosis_vin.sample(n=514, random_state=9)

In [50]:
Indeterminate_extra = pd.concat([ILD_vin, Atelectasis_vin, Infiltration_vin, Pulmonary_fibrosis_vin])
Indeterminate_extra.shape

(1806, 3)

In [51]:
Indeterminate_extra['Indeterminate Appearance'] = 1
Indeterminate_extra['sparse_gt'] = 2
Indeterminate_extra.head()

,image_id,class_name,dataset,Indeterminate Appearance,sparse_gt
5530,6a5a92798a840e7b12533ada2e2816cb,ILD,vinbig,1,2
6363,d617b24dad98aa3b1f2d645828095355,ILD,vinbig,1,2
36269,0e88612bd294e249382b9e64d222cdf4,ILD,vinbig,1,2
38616,cfe3e67ca5f8235d64e51416a26e70cc,ILD,vinbig,1,2
20256,fdd323a08a8d9890c938351d465108b6,ILD,vinbig,1,2


## Getting Negative Data (External Sources (NIH and VinBig))

In [52]:
needed_samples = 2855
we_have = 1676
extra_needed = needed_samples - we_have
extra_needed

1179

In [53]:
Negative_vin = vin_labels[vin_labels.class_name == 'No finding']
Negative_vin.shape

(31818, 3)

In [54]:
Negative_extra = Negative_vin.sample(n=1179, random_state=9)
Negative_extra.shape

(1179, 3)

In [55]:
Negative_extra['Negative for Pneumonia'] = 1
Negative_extra['sparse_gt'] = 0
Negative_extra.head()

,image_id,class_name,dataset,Negative for Pneumonia,sparse_gt
12423,63246dd75fb16f49acec60fb00c247da,No finding,vinbig,1,0
6832,e29fcb9b89f4ccf3e2caeb609b99fb5f,No finding,vinbig,1,0
65447,e071243215a835dbf5f53328856f4126,No finding,vinbig,1,0
5046,874317c4604b813be6e377b963b50583,No finding,vinbig,1,0
34649,ca6c4552248da1513a23759b61e22f65,No finding,vinbig,1,0


In [56]:
# Negative_nih = nih_labels[nih_labels['Finding Labels'] == 'No Finding']
# Negative_nih.shape

# Making the Full Dataset

In [57]:
train_df.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt
0,000a312787f2,0,1,0,0,1
1,000c3a3f293f,1,0,0,0,0
2,0012ff7358bc,0,1,0,0,1
3,001398f4ff4f,0,0,0,1,3
4,001bd15d1891,0,1,0,0,1


In [58]:
Atypical_extra.head()

,image_id,class_name,dataset,Atypical Appearance,sparse_gt
212,6e4391555899c8474c4d32f42b2ba21b,Pneumothorax,vinbig,1,3
308,e3c5ff1eb0d55039f3bbc3c4875543c3,Pneumothorax,vinbig,1,3
1090,54e6184c63c75a9695d7effc17969ad0,Pneumothorax,vinbig,1,3
1140,71511f823c4664475131c1519d9bd0a8,Pneumothorax,vinbig,1,3
1440,f9a0f4bd4bb3553af6e939362bbba5d6,Pneumothorax,vinbig,1,3


In [59]:
train_df.columns = ['image_id', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance', 'sparse_gt']

In [60]:
train_df.head()

,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt
0,000a312787f2,0,1,0,0,1
1,000c3a3f293f,1,0,0,0,0
2,0012ff7358bc,0,1,0,0,1
3,001398f4ff4f,0,0,0,1,3
4,001bd15d1891,0,1,0,0,1


In [61]:
train_df['dataset'] = 'siim'

In [62]:
extra_df = pd.concat([Atypical_extra, Indeterminate_extra, Negative_extra])

In [63]:
extra_df.head()

,image_id,class_name,dataset,Atypical Appearance,sparse_gt,Indeterminate Appearance,Negative for Pneumonia
212,6e4391555899c8474c4d32f42b2ba21b,Pneumothorax,vinbig,1.0,3,NaN,NaN
308,e3c5ff1eb0d55039f3bbc3c4875543c3,Pneumothorax,vinbig,1.0,3,NaN,NaN
1090,54e6184c63c75a9695d7effc17969ad0,Pneumothorax,vinbig,1.0,3,NaN,NaN
1140,71511f823c4664475131c1519d9bd0a8,Pneumothorax,vinbig,1.0,3,NaN,NaN
1440,f9a0f4bd4bb3553af6e939362bbba5d6,Pneumothorax,vinbig,1.0,3,NaN,NaN


# Data Generator

In [64]:
import os

import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold

In [65]:
SATURATION  = (0.9, 1.1)
CONTRAST = (0.9, 1.1)
BRIGHTNESS  =  0.1
ROTATION    = 10.0
SHEAR    = 2.0
HZOOM  = 8.0
WZOOM  = 4.0
HSHIFT = 4.0
WSHIFT = 4.0

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = 640
    XDIM = DIM%2
    
    rot = ROTATION * tf.random.normal([1], dtype='float32')
    shr = SHEAR * tf.random.normal([1], dtype='float32')
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM
    h_shift = HSHIFT * tf.random.normal([1], dtype='float32')
    w_shift = WSHIFT * tf.random.normal([1], dtype='float32')
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot, shr, h_zoom, w_zoom, h_shift, w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]), label

In [66]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_saturation(img, SATURATION[0], SATURATION[1])
        img = tf.image.random_contrast(img, CONTRAST[0], CONTRAST[1])
        img = tf.image.random_brightness(img, BRIGHTNESS)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.map(transform, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [67]:
COMPETITION_NAME1 = "vinbigdata-chest-xray-original-png"
COMPETITION_NAME2 = "covid19-detection-890pxpng-study"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
GCS_DS_PATH1 = KaggleDatasets().get_gcs_path(COMPETITION_NAME1)
GCS_DS_PATH2 = KaggleDatasets().get_gcs_path(COMPETITION_NAME2)

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [68]:
extra_df['image_id'] = GCS_DS_PATH1 + '/train/' + extra_df['image_id'] + '.png'
train_df['image_id'] = GCS_DS_PATH2 + '/train/' + train_df['image_id'] + '.png'

In [69]:
final_df = pd.concat([train_df, extra_df])
final_df.head()

,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt,dataset,class_name
0,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0.0,1.0,0.0,0.0,1,siim,NaN
1,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,1.0,0.0,0.0,0.0,0,siim,NaN
2,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0.0,1.0,0.0,0.0,1,siim,NaN
3,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0.0,0.0,0.0,1.0,3,siim,NaN
4,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0.0,1.0,0.0,0.0,1,siim,NaN


In [70]:
final_df.iloc[:,0][0]

'gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5d84e1092b52a708d805/train/000a312787f2.png'

In [71]:
final_df.fillna(0, inplace=True)

In [72]:
label_cols = final_df.columns[1:5]
label_cols

Index(['Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance'],
      dtype='object')

In [73]:
for i in label_cols:
    final_df[i] = final_df[i].astype(int)
final_df.head()

,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt,dataset,class_name
0,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1,siim,0
1,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,1,0,0,0,0,siim,0
2,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1,siim,0
3,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,0,0,1,3,siim,0
4,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1,siim,0


In [74]:
final_df.sum()

image_id                    gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...
Negative for Pneumonia                                                   2855
Typical Appearance                                                       2855
Indeterminate Appearance                                                 2855
Atypical Appearance                                                      2855
sparse_gt                                                               17130
dataset                     siimsiimsiimsiimsiimsiimsiimsiimsiimsiimsiimsi...
dtype: object

In [75]:
final_df = final_df.iloc[:, 0:6]
final_df.head()

,image_id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sparse_gt
0,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1
1,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,1,0,0,0,0
2,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1
3,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,0,0,1,3
4,gs://kds-fc3be7d2ac2afce0ec3cbdec79b75f7fb2ad5...,0,1,0,0,1


In [76]:
#from sklearn.model_selection import train_test_split

#X_train, X_valid = train_test_split(final_df, test_size=1800, random_state=9, stratify = final_df['sparse_gt'])

In [77]:
final_df.reset_index(inplace = True)

In [78]:
gkf  = GroupKFold(n_splits = 5)
final_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(final_df, groups = final_df.image_id.tolist())):
    final_df.loc[val_idx, 'fold'] = fold

In [79]:
#X_train.sparse_gt.value_counts()

In [80]:
#X_valid.sparse_gt.value_counts()

In [81]:
for i in range(5):
     
    valid_paths = final_df[final_df['fold'] == i]['image_id'] #"/train/"
    train_paths = final_df[final_df['fold'] != i]['image_id']  #"/train/" 
    valid_labels = final_df[final_df['fold'] == i][label_cols].values
    train_labels = final_df[final_df['fold'] != i][label_cols].values
    #valid_paths = X_valid['image_id'] #"/train/"
    #train_paths = X_train['image_id'] #"/train/" 
    #valid_labels = X_valid[label_cols].values
    #train_labels = X_train[label_cols].values

    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 640)
    IMS = 7

    decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
        repeat=False, shuffle=False, augment=False
    )

    try:
        n_labels = train_labels.shape[1]
    except:
        n_labels = 1

    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(n_labels, activation='softmax')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])

        model.summary()


    steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'model{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

    history = model.fit(
        train_dataset, 
        epochs=30,
        verbose=1,
        callbacks=[checkpoint, lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f'history{i}.csv')

258441216/258434480 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 20, 20, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 10244     
Total params: 64,107,924
Trainable params: 63,797,204
Non-trainable params: 310,720
_________________________________________________________________
Epoch 1/30
71/71 [==============================] - 660s 6s/step - loss: 1.1316 - auc: 0.6947 - val_loss: 5.0733 - val_auc: 0.6354
Epoch 2/30
71/71 [==============================] - 133s 2s/step - loss: 1.0153 - auc: 0.7693 - val_loss: 8.6243 - val_auc: 0.5494
Epoch 3/30
71/71 [=====================